In [1]:
import numpy as np
import pandas as pd

import sqlite3

# 1. Lưu và đọc DataFrame vào/từ database

## 1.1 Tạo ra bảng HocVien từ DataFrame

In [2]:
df_hocvien = pd.DataFrame({'name':['Lan', 'Mai', 'Phong'], 'age':[24, 22, 25]})
#df_test['id'] = df_test.index
df_hocvien

,name,age
0,Lan,24
1,Mai,22
2,Phong,25


**Kết nối CSDL và lưu lại**

In [3]:
# Kết nối đến CSDL
conn = sqlite3.connect('database/quan_ly_dao_tao.db')

In [4]:
# Sử dụng hàm df.to_sql()
# Lưu ý: name ==> tên của bảng
df_hocvien.to_sql(name='hocvien', con=conn, if_exists='replace')

# Cần thực hiện commit()
conn.commit()

**Thử tạo `cursor` để truy vấn**

In [5]:
cursor = conn.cursor()
query = """
        SELECT *
        FROM hocvien
"""
cursor.execute(query)
pd.DataFrame(cursor.fetchall(), columns=[item[0] for item in cursor.description])

,index,name,age
0,0,Lan,24
1,1,Mai,22
2,2,Phong,25


## 1.2. Đọc thẳng từ DB lên dataframe

In [ ]:
# Nếu chưa có kết nối đến CSDL
# conn = sqlite3.connect('database/quan_ly_dao_tao.db')

In [6]:
df1 = pd.read_sql("SELECT * from hocvien", conn)

In [7]:
df1

,index,name,age
0,0,Lan,24
1,1,Mai,22
2,2,Phong,25


## 1.3. Tạo bảng MonHoc từ DataFrame

In [8]:
df_monhoc = pd.DataFrame({'name':['Lập Trình Python', 'Machine Learning', 'Deep Learning'], 
                          'weeks':[10, 12, 12]})
df_monhoc

,name,weeks
0,Lập Trình Python,10
1,Machine Learning,12
2,Deep Learning,12


In [9]:
df_monhoc.to_sql(name='monhoc', con=conn, if_exists='replace')
conn.commit()

In [10]:
# Thử truy vấn dữ liệu
pd.read_sql("SELECT * from monhoc", conn)

,index,name,weeks
0,0,Lập Trình Python,10
1,1,Machine Learning,12
2,2,Deep Learning,12


# 2. Kết hợp pandas và SQL cho truy vấn dữ liệu

## 2.1. Sử dụng hàm `df.query()`

(Dữ liệu instacart)

In [11]:
df_product = pd.read_csv('data-instacart/products.csv', index_col='product_id')
df_product.head()

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13


**Cú pháp `df.query()` cơ bản**

In [12]:
aisle_muon_tim = 1
df_product.query("aisle_id == %d"%(aisle_muon_tim))

,product_name,aisle_id,department_id
product_id,,,
209,Italian Pasta Salad,1,20
554,Turkey Chili,1,20
886,Whole Grain Salad with Roasted Pecans & Mango ...,1,20
1600,Mediterranean Orzo Salad,1,20
2539,Original Potato Salad,1,20
...,...,...,...
47979,Butternut Squash Bisque,1,20
48474,Deli Fresh Coleslaw,1,20
48556,Red Potato Salad,1,20


In [13]:
# Tương tự như dùng cách truy cập chỉ số
df_product[df_product['aisle_id'] == aisle_muon_tim]

,product_name,aisle_id,department_id
product_id,,,
209,Italian Pasta Salad,1,20
554,Turkey Chili,1,20
886,Whole Grain Salad with Roasted Pecans & Mango ...,1,20
1600,Mediterranean Orzo Salad,1,20
2539,Original Potato Salad,1,20
...,...,...,...
47979,Butternut Squash Bisque,1,20
48474,Deli Fresh Coleslaw,1,20
48556,Red Potato Salad,1,20


**Cú pháp trong query: điều kiện trên các cột**

In [14]:
df_product.query("department_id >= 3 and department_id <=8")

,product_name,aisle_id,department_id
product_id,,,
3,Robust Golden Unsweetened Oolong Tea,94,7
7,Pure Coconut Water With Orange,98,7
10,Sparkling Orange Juice & Prickly Pear Beverage,115,7
11,Peach Mango Juice,31,7
20,Pomegranate Cranberry & Aloe Vera Enrich Drink,98,7
...,...,...,...
49682,California Limeade,98,7
49683,Cucumber Kirby,83,4
49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5


In [15]:
# Giả sử cần so sánh các giá trị giữa 2 cột
df_product.query("aisle_id < department_id/5 and aisle_id > department_id/7")

,product_name,aisle_id,department_id
product_id,,,
25,Salted Caramel Lean Protein & Fiber Bar,3,19
77,Coconut Chocolate Chip Energy Bar,3,19
164,Peanut Butter Cacao Quinoa Clusters,3,19
183,Cookie Dough Performance Energy,3,19
205,Chocolate Mint Cookie Crunch Nutrition Bar,3,19
...,...,...,...
49415,All Natural Cookies & Cream Muscle Brownie,3,19
49525,Cinnamon Roll Flavor Protein Bar,3,19
49561,Crunchy Peanut Butter Bars,3,19


## 2.2. Sử dụng module `pandasql`

**Cần cài đặt nếu chưa có sẵn**

In [ ]:
%pip install -U pandasql

In [16]:
import pandasql as pdsql
from pandasql import sqldf    # Hàm sqldf
from pandasql import PandaSQL # Lớp PandaSQL
print(pdsql)
print(sqldf)

<module 'pandasql' from 'D:\\tqkhai\\DevEnv\\anaconda3\\lib\\site-packages\\pandasql\\__init__.py'>
<function sqldf at 0x000001707200A790>


**Sử dụng hàm `sqldf`**

In [17]:
df_product

,product_name,aisle_id,department_id
product_id,,,
1,Chocolate Sandwich Cookies,61,19
2,All-Seasons Salt,104,13
3,Robust Golden Unsweetened Oolong Tea,94,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
5,Green Chile Anytime Sauce,5,13
...,...,...,...
49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49685,En Croute Roast Hazelnut Cranberry,42,1
49686,Artisan Baguette,112,3


In [18]:
# Sử dụng tham chiếu đến "globals()" hoặc "locals()"
query = """
        SELECT * 
        FROM df_product 
        WHERE aisle_id=100 
        LIMIT 10 
        -- OFFSET 5
"""
pdsql.sqldf(query, env=globals())

,product_id,product_name,aisle_id,department_id
0,38,Ultra Antibacterial Dish Liquid,100,21
1,72,Organic Honeycrisp Apples,100,21
2,110,Uncured Turkey Bologna,100,21
3,297,"Write Bros Ball Point Pens, Cap-Pen, Medium (1...",100,21
4,417,Classics Baby Binks Easter Chocolate Bunny,100,21
5,437,Strawberry Cheesecake Nonfat Yogurt,100,21
6,440,Crossovers Olive Thyme Almond,100,21
7,472,Fancy Diced Hot Chilies,100,21
8,491,Pompelmo Water,100,21
9,556,Bake & Break Cheese Garlic Loaf,100,21


**Trường hợp chỉ sử dụng được `locals()`**

In [19]:
query = """
        SELECT department_id, count(aisle_id) as n_aisles
        FROM df_product1
        GROUP BY department_id
        ORDER BY n_aisles DESC
        LIMIT (10)
"""

def ThuNghiem(df):
    df_product1 = df.copy()
    return sqldf(query, env=locals())

ThuNghiem(df_product)

,department_id,n_aisles
0,11,6563
1,19,6264
2,13,5371
3,7,4365
4,1,4007
5,16,3449
6,17,3085
7,15,2092
8,9,1858
9,4,1684


**Được khuyến cáo nên sử dụng lớp PandaSQL**

In [20]:
# (1) Tạo đối tượng PandaSQL
pdsql2 = PandaSQL()

In [21]:
# (2) Query trong 1 hàm nào đó với tham số locals()
pdsql2("SELECT aisle_id, product_id FROM df_product WHERE department_id=3 LIMIT 10 OFFSET 5", locals())

,aisle_id,product_id
0,93,131
1,112,189
2,8,218
3,93,245
4,112,317
5,112,346
6,112,444
7,43,517
8,112,566
9,128,595


In [22]:
# (3) Query toàn cục với tham số globals()
pdsql2("SELECT aisle_id, product_id FROM df_product WHERE department_id=3 GROUP BY aisle_id", globals())

,aisle_id,product_id
0,8,218
1,43,517
2,93,116
3,112,101
4,128,58


## 2.3. Truy vấn từ nhiều dataframe

In [23]:
df_aisle = pd.read_csv('data-instacart/aisles.csv', index_col='aisle_id')
df_aisle.head()

,aisle
aisle_id,
1,prepared soups salads
2,specialty cheeses
3,energy granola bars
4,instant foods
5,marinades meat preparation


**Kết hợp các thông tin từ `products.csv` và `aisles.csv`**

In [24]:
query = """
        SELECT p.aisle_id, count(p.product_id) as n_products, a.aisle
        FROM df_product as p
        LEFT JOIN df_aisle as a
            ON a.aisle_id = p.aisle_id
        GROUP BY p.aisle_id
        ORDER BY n_products DESC
        LIMIT (10)
"""

pdsql2(query, env=locals())

,aisle_id,n_products,aisle
0,100,1258,missing
1,45,1246,candy chocolate
2,37,1091,ice cream ice
3,47,1038,vitamins supplements
4,120,1026,yogurt
5,107,989,chips pretzels
6,94,894,tea
7,21,891,packaged cheese
8,38,880,frozen meals
9,61,874,cookies cakes


**Lấy tên của các sản phẩm và aisle tương ứng cho department số 3**

In [25]:
query = """ SELECT p.product_name, a.aisle
            FROM df_product p
            INNER JOIN df_aisle a
            ON p.aisle_id = a.aisle_id
            WHERE p.department_id = 3
            """
pdsql2(query, globals())

,product_name,aisle
0,Whole Wheat Tortillas,tortillas flat bread
1,100% Whole Wheat Pita Bread,tortillas flat bread
2,"Bread, Healthy Whole Grain",bread
3,Organic Sprouted Barley Bread,bread
4,English Muffins,breakfast bakery
...,...,...
1511,Gluten Free Classic Sourdough,bread
1512,Challah Bread,bread
1513,Sourdough Loaf,bread
1514,Masada Kosher Pocket Bread,tortillas flat bread


**Lấy ra 10 aisle có nhiều loại sản phẩm nhất**

In [26]:
query = """ SELECT a.aisle, COUNT(p.product_name) AS NProducts
            FROM df_product AS p
            LEFT JOIN  df_aisle a
            ON a.aisle_id = p.aisle_id
            GROUP BY a.aisle
            ORDER BY NProducts Desc
            LIMIT 10
            """
pdsql2(query, globals())

,aisle,NProducts
0,missing,1258
1,candy chocolate,1246
2,ice cream ice,1091
3,vitamins supplements,1038
4,yogurt,1026
5,chips pretzels,989
6,tea,894
7,packaged cheese,891
8,frozen meals,880
9,cookies cakes,874


**Lấy ra tên của các department tương ứng**

In [27]:
df_department = pd.read_csv('data-instacart/departments.csv', index_col='department_id')
df_department.head()

,department
department_id,
1,frozen
2,other
3,bakery
4,produce
5,alcohol


In [28]:
# Query dữ liệu từ 3 bảng (dataframe)
query = """ SELECT p.product_name, a.aisle, d.department
            FROM df_product p
            JOIN df_aisle a
                ON p.aisle_id = a.aisle_id
            JOIN df_department d
                ON d.department_id = p.department_id
            """
pdsql2(query, globals())

,product_name,aisle,department
0,Chocolate Sandwich Cookies,cookies cakes,snacks
1,All-Seasons Salt,spices seasonings,pantry
2,Robust Golden Unsweetened Oolong Tea,tea,beverages
3,Smart Ones Classic Favorites Mini Rigatoni Wit...,frozen meals,frozen
4,Green Chile Anytime Sauce,marinades meat preparation,pantry
...,...,...,...
49683,"Vodka, Triple Distilled, Twist of Vanilla",spirits,alcohol
49684,En Croute Roast Hazelnut Cranberry,frozen vegan vegetarian,frozen
49685,Artisan Baguette,bread,bakery
49686,Smartblend Healthy Metabolism Dry Cat Food,cat food care,pets


In [29]:
query = """ SELECT d.department, a.aisle, count(p.product_name) as n_products
            FROM df_product p
            JOIN df_aisle a
                ON p.aisle_id = a.aisle_id
            JOIN df_department d
                ON d.department_id = p.department_id
            GROUP BY d.department, a.aisle
            """
pdsql2(query, globals())

,department,aisle,n_products
0,alcohol,beers coolers,385
1,alcohol,red wines,232
2,alcohol,specialty wines champagnes,95
3,alcohol,spirits,195
4,alcohol,white wines,147
...,...,...,...
129,snacks,ice cream toppings,85
130,snacks,mint gum,168
131,snacks,nuts seeds dried fruit,582
132,snacks,popcorn jerky,316
